In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50     # steepness factor
num_blocks = 10000 # number of expression blocks
num_data = 1900    # number of data
test_size = 100    # number of test size
val_size = 1000    # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 2000                   # number of training epochs
warmup = 400                    # number of epochs to wait before enacting early stopping policy
patience = 200                  # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2593960.00
Epoch 1, Validation Loss: 573909356.00
Epoch 2, Validation Loss: 3488881868800.00
Epoch 3, Validation Loss: 126014059520.00
Epoch 4, Validation Loss: 123627577.50
Epoch 5, Validation Loss: 5490150.59
Epoch 6, Validation Loss: 3705841.02
Epoch 7, Validation Loss: 2012055.21
Epoch 8, Validation Loss: 1554858.25
Epoch 9, Validation Loss: 1706727.34
Epoch 10, Validation Loss: 1734503.23
Epoch 11, Validation Loss: 2040054.13
Epoch 12, Validation Loss: 1453294.98
Epoch 13, Validation Loss: 1293839.89
Epoch 14, Validation Loss: 1321966.72
Epoch 15, Validation Loss: 1050613.75
Epoch 16, Validation Loss: 873178.72
Epoch 17, Validation Loss: 1652335.52
Epoch 18, Validation Loss: 1588407.82
Epoch 19, Validation Loss: 1129695.05
Epoch 20, Validation Loss: 1730939.39
Epoch 21, Validation Loss: 1810683.86
Epoch 22, Validation Loss: 884861.55
Epoch 23, Validation Loss: 1596195.88
Epoch 24, Validation Loss: 1387324.20
Epoch 25, Validation Loss: 919176.92
Epoch 26, 

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-10000_s.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:13<00:00,  1.33s/it]


             Obj Val  Constraints Viol  Elapsed Time
count     100.000000        100.000000    100.000000
mean    71153.106690       3494.335871      0.044617
std     41521.282294       5338.217303      0.004960
min     29406.999489          0.000000      0.010635
25%     39667.823104          0.000000      0.044043
50%     62969.347254          0.000000      0.044551
75%     74510.622683       5494.146586      0.046336
max    202908.713203      20595.257874      0.050940
Number of infeasible solution: 47


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 1024          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 2000                   # number of training epochs
warmup = 400                    # number of epochs to wait before enacting early stopping policy
patience = 200                  # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs=epochs, patience=patience, warmup=warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 2346129.37
Epoch 1, Validation Loss: 2901645520.00
Epoch 2, Validation Loss: 520412239872.00
Epoch 3, Validation Loss: 441771270144.00
Epoch 4, Validation Loss: 14536662.56
Epoch 5, Validation Loss: 3396917.30
Epoch 6, Validation Loss: 3139643.17
Epoch 7, Validation Loss: 25771237.62
Epoch 8, Validation Loss: 1617424.49
Epoch 9, Validation Loss: 942228.27
Epoch 10, Validation Loss: 925896.19
Epoch 11, Validation Loss: 839255.34
Epoch 12, Validation Loss: 1338706.82
Epoch 13, Validation Loss: 1627481.77
Epoch 14, Validation Loss: 833669.55
Epoch 15, Validation Loss: 1223276.12
Epoch 16, Validation Loss: 570126.07
Epoch 17, Validation Loss: 1412495.75
Epoch 18, Validation Loss: 1928513.25
Epoch 19, Validation Loss: 807339.38
Epoch 20, Validation Loss: 698801.66
Epoch 21, Validation Loss: 1538501.50
Epoch 22, Validation Loss: 939586.76
Epoch 23, Validation Loss: 523234.65
Epoch 24, Validation Loss: 1209823.64
Epoch 25, Validation Loss: 837182.74
Epoch 26, Validat

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-10000_s.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:26<00:00,  1.46s/it]


             Obj Val  Constraints Viol  Elapsed Time
count     100.000000        100.000000    100.000000
mean    72642.517708       3400.201152      0.044910
std     16300.099220       4819.339689      0.006332
min     46607.425245          0.000000      0.013127
25%     59332.225938          0.000000      0.044367
50%     67825.095512          0.000000      0.045351
75%     82466.780814       6212.425504      0.047454
max    113351.694185      17974.054885      0.053854
Number of infeasible solution: 48
